ideia: usar xgboost + output to yolo do modelo08 + metadata das images para fazer a contagem + rbg

razao: corrigir erros sistematicos do modelo08 que possam ser causados qnd a imagem é ... *metadata* ...

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
import xgboost as xgb
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [34]:
df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

df_train.sample(1)

,img_id,img_origin,img_placement,nr_boil,polygons_boil,nr_pan,polygons_pan
1445,IDiannr,S,S-unknown,[1],"[[(247.1167192429022, 342.61654135338347), (16...",[],[]


In [ ]:


def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["img_placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["img_placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["img_placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["img_placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["img_placement"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [23]:
df_train.head()

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
0,IDUXSjTNlUQ59,[],[],"[14, 16]","[[(228.0551181102362, 277.6509186351706), (231...","[0, 0, 1, 0, 1, 0]"
1,IDln9t3L,[],[],[2],"[[(263.552, 259.584), (316.032, 237.056), (344...","[1, 0, 0, 0, 1, 0]"
2,IDdLAJld1z,"[1, 1, 1, 1]","[[(313.9878787878788, 384.1294236602629), (339...",[],[],"[1, 0, 0, 0, 1, 0]"
3,ID6g6IV,[],[],[2],"[[(246.784, 171.17866666666666), (283.648, 156...","[1, 0, 0, 0, 1, 0]"
4,IDrxyCyN,[1],"[[(203.37777777777777, 235.99198396793588), (1...",[],[],"[0, 0, 0, 1, 0, 1]"


In [24]:
df_val.head()

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
0,IDIyeJf,[],[],[1],"[[(184.32, 243.712), (201.728, 193.87733333333...","[1, 0, 0, 0, 1, 0]"
1,IDIuxfUrw8kOOu4DI,[],[],[1],"[[(227.456, 202.752), (257.408, 163.328), (289...","[1, 0, 0, 0, 1, 0]"
2,IDGYlhoJdU,[],[],"[2, 1]","[[(192.256, 225.28), (224.256, 230.4), (220.54...","[1, 0, 0, 0, 1, 0]"
3,IDUpWDxrBZj,"[1, 1]","[[(266.82910321489004, 288.89385474860336), (2...",[],[],"[0, 0, 0, 1, 0, 1]"
4,IDOaq1s,[1],"[[(355.6848484848485, 156.86147623862487), (36...",[],[],"[1, 0, 0, 0, 1, 0]"


In [25]:
from ultralytics import YOLO

model = YOLO("ModelBuild/model08_yoloJ/weights/best.pt")

In [26]:
# Load EfficientNetB0 without the top layer
effnet = EfficientNetB5(weights="imagenet", include_top=False, pooling="avg")

datagen = ImageDataGenerator(
    rotation_range=40,       # Random rotation range in degrees
    width_shift_range=0.2,   # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,         # Random shear transformation
    horizontal_flip=True,    # Random horizontal flip
    fill_mode='nearest',      # How to fill in newly created pixels (nearest pixel)
    brightness_range=[0.2, 1.8],
    
)

# Function to extract features from an image
def extract_image_features(image_path):

    image_path = "images/" + image_path + ".jpg"

    img = cv2.imread(image_path)
    img = cv2.resize(img, (456, 456))
    img = preprocess_input(img)  # Normalize image
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    augmented_img = datagen.random_transform(img[0].copy())
    augmented_img = np.expand_dims(augmented_img, axis=0)  # Add batch dimension


    features = effnet.predict(augmented_img, verbose=0)  # Extract features
    return features.flatten()  # Convert to 1D


# bypass waiting time
import pickle
with open('ModelBuild/model06_data.pkl', 'rb') as f:
    a = pickle.load(f)

In [31]:
X_train = a["X_train"]
X_val = a["X_val"]
y_train = a["y_train"]
y_val = a["y_val"]

---
---

preparar features

In [27]:

def extract_yolo_features(df, path, image_col = "img_id"):
    yolo_features = []
    for i, row in df.iterrows():
        if i % 100 == 0:
            print(i, len(df))

        #resize_image(row["ID"])
        results = model(f"{path}/{row[image_col]}.png", verbose=False)
        result = results[0]

        class_counts = {}

        for cls in result.boxes.cls.tolist():
            class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1

        class_names = model.names
        named_counts = {class_names[k]: v for k, v in class_counts.items()}

        if "boil_alone" in named_counts:
            boils = named_counts["boil_alone"]
        else:
            boils = 0
        
        if "pan_alone" in named_counts:
            pans = named_counts["pan_alone"]
        else:
            pans = 0

        yolo_features.append([boils, pans])
    
    df["yolo_features"] = yolo_features

#extract_yolo_features(df_val, "ModelBuild/model08/val/images")
#extract_yolo_features(df_train, "ModelBuild/model08/train/images")

In [28]:
import pickle
#df_train.to_pickle("ModelBuild/model10/Model_Train.pkl")
#df_val.to_pickle("ModelBuild/model10/Model_Val.pkl")

df_train = pd.read_pickle("ModelBuild/model10/Model_Train.pkl")
df_val = pd.read_pickle("ModelBuild/model10/Model_Val.pkl")

df_train.head(1)

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features,yolo_features
0,IDUXSjTNlUQ59,[],[],"[14, 16]","[[(228.0551181102362, 277.6509186351706), (231...","[0, 0, 1, 0, 1, 0]","[0, 1]"


---
---
---

xgboost

In [29]:
X_train = df_train[["more_features", "yolo_features"]]
X_train = np.array([row["more_features"] + row["yolo_features"] for i, row in X_train.iterrows()])
y_train = df_train[["nr_boil", "nr_pan"]]
y_train = np.array([ [sum(row["nr_boil"]) , sum(row["nr_pan"])] for i, row in y_train.iterrows()])

X_val = df_val[["more_features", "yolo_features"]]
X_val = np.array([row["more_features"] + row["yolo_features"] for i, row in X_val.iterrows()])
y_val = df_val[["nr_boil", "nr_pan"]]
y_val = np.array([ [sum(row["nr_boil"]) , sum(row["nr_pan"])] for i, row in y_val.iterrows()])


In [30]:
xgb_regressor = MultiOutputRegressor(xgb.XGBRegressor(objective="reg:squarederror",
                                                      random_state=42))

param_dist = {
    "estimator__n_estimators": [100, 300, 500, 700],  # More trees improve results
    "estimator__learning_rate": [0.005, 0.01, 0.05],  # Lower LR prevents overfitting
    "estimator__max_depth": [3, 5, 7],  # Limit tree depth for large feature sets
    "estimator__subsample": [0.4, 0.5, 0.7, 0.9],  # Row sampling
    "estimator__colsample_bytree": [0.3, 0.5, 0.6],  # Feature sampling (important for 2K features)
    "estimator__reg_lambda": [5, 10, 20, 50],  # L2 Regularization
    "estimator__reg_alpha": [5, 10, 50],  # L1 Regularization
    "estimator__gamma": [0, 0.1, 0.2, 0.5],  # Controls tree split sensitivity
}

# 5. Run RandomizedSearchCV
random_search = RandomizedSearchCV(xgb_regressor, param_distributions=param_dist, 
                                   n_iter=10, scoring="neg_mean_absolute_error",
                                   cv=7, verbose=2, n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)

Fitting 7 folds for each of 10 candidates, totalling 70 fits


Traceback (most recent call last):
  File "/opt/anaconda3/envs/CAA01/lib/python3.10/runpy.py", line 187, in _run_module_as_main
Traceback (most recent call last):
  File "/opt/anaconda3/envs/CAA01/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/anaconda3/envs/CAA01/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/opt/anaconda3/envs/CAA01/lib/python3.10/site-packages/joblib/__init__.py", line 114, in <module>
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/anaconda3/envs/CAA01/lib/python3.10/runpy.py", line 110, in _get_module_details
    from .memory import Memory
  File "/opt/anaconda3/envs/CAA01/lib/python3.10/site-packages/joblib/memory.py", line 12, in <module>
    __import__(pkg_name)
  File "/opt/anaconda3/envs/CAA01/lib/python3.10/site-packages/joblib/__init__.py", line 114, in <module>
    from .memory import 

KeyboardInterrupt: 

In [ ]:
best_params = random_search.best_params_
best_params

{'estimator__subsample': 0.7,
 'estimator__reg_lambda': 20,
 'estimator__reg_alpha': 5,
 'estimator__n_estimators': 15,
 'estimator__max_depth': 5,
 'estimator__learning_rate': 0.05,
 'estimator__gamma': 0.5,
 'estimator__colsample_bytree': 1}

In [ ]:
best_model = random_search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_val)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_val, y_pred)
print(f"Test MAE: {mae}")

y_pred = best_model.predict(X_train)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_train, y_pred)
print(f"Train MAE: {mae}")

Test MAE: 2.2065573770491804
Train MAE: 1.9368593685936861


# submission

In [ ]:
submission = pd.read_csv("zindi_files/SampleSubmission.csv")
test = pd.read_csv("Test.csv")

def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["placement"], inplace=True)

#hot_encod_img_placement(test)
#extract_yolo_features(test, "ModelBuild/model08_subm", image_col = "ID")
#test.to_pickle("ModelBuild/model10/Test.pkl")
test = pd.read_pickle("ModelBuild/model10/Test.pkl")

In [ ]:
test

,ID,more_features,yolo_features
0,ID00qprY,"[1, 0, 0, 0, 1, 0]","[0, 1]"
1,ID01AciUc,"[1, 0, 0, 0, 1, 0]","[0, 1]"
2,ID0328D,"[1, 0, 0, 0, 1, 0]","[0, 2]"
3,ID05WxObCFTs9,"[1, 0, 0, 0, 1, 0]","[0, 2]"
4,ID06AdCmLMlkO,"[0, 0, 0, 1, 0, 1]","[1, 0]"
...,...,...,...
1102,IDzhpdPHeGHlmWEIx,"[1, 0, 0, 0, 1, 0]","[0, 1]"
1103,IDztljx,"[1, 0, 0, 0, 1, 0]","[0, 1]"
1104,IDzwPcyO0e,"[0, 0, 0, 1, 0, 1]","[8, 1]"
1105,IDzzNegz,"[1, 0, 0, 0, 1, 0]","[0, 1]"


In [ ]:
X_test = test[["more_features", "yolo_features"]]
X_test = np.array([row["more_features"] + row["yolo_features"] for i, row in X_test.iterrows()])

In [ ]:
X_test

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 2],
       ...,
       [0, 0, 0, ..., 1, 8, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 3]])

In [ ]:
pred = best_model.predict(X_test)
for i, row in test.iterrows():
    submission.loc[submission["ID"] == f"{row['ID']}_boil", "Target"] = pred[i][0]
    submission.loc[submission["ID"] == f"{row['ID']}_pan", "Target"] = pred[i][1]

submission.head()

/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_86472/4204815357.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.11831264197826385' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission.loc[submission["ID"] == f"{row['ID']}_boil", "Target"] = pred[i][0]


,ID,Target
0,ID00qprY_boil,0.118313
1,ID00qprY_pan,3.117506
2,ID01AciUc_boil,0.118313
3,ID01AciUc_pan,3.117506
4,ID0328D_boil,0.118313


In [ ]:
submission.to_csv("zindi_files/model10_submission.csv", index=False)